# Exploração de dados: EDA e pré-processamento
Giulia Chimini Stefainski, Leonardo Azzi Martins, Matheus de Moraes Costa

---

**Objetivo:** realizar uma análise exploratória de dados, e a partir disto definir possibilidades de pré-processamento para o dataset.

# Setup

In [9]:
%pip install pandas==1.5.3 transformers==4.50.2 datasets==3.5.0 scikit-learn==1.4.2 evaluate==0.4.3 seaborn==0.13.2 imblearn accelerate==1.5.2 emoji==2.14.1 torch==2.6.0
#pip install torch==2.6.0+cu124 Funciona no databricks, para Torch com CUDA. Veja aí pra sua máquina

  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached transformers-4.50.2-py3-none-any.whl (10.2 MB)
  Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
  Using cached scikit_learn-1.4.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached accelerate-1.5.2-py3-none-any.whl (345 kB)
  Using cached emoji-2.14.1-py3-none-any.whl (590 kB)
  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl (766.7 MB)
  Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 1.3 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl (56.3 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.

In [1]:
from transformers import Trainer
from datasets import Dataset
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
import emoji

/home/leonardoazzi/cic/25-1/pln/trabalho/bert-misinfo-covid19/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'emoji'

# Preparação de dados
Carrega o dataset a ser utilizado para fine-tuning e seleciona os atributos mais relevantes.

Faz o download do dataset anotado no diretório ./data

In [2]:
import os

if not os.path.exists('./data/covidbr_labeled.csv'):
  %mkdir data
  %curl -L -o ./data/covidbr_labeled.csv https://zenodo.org/records/5193932/files/covidbr_labeled.csv
else:
    print("File already exists. Skipping download.")

File already exists. Skipping download.


In [3]:
original_dataset_df = pd.read_csv('./data/covidbr_labeled.csv')
original_dataset_df

,shares,text,misinformation,source,revision
0,27,"O ministro da Ciência, Tecnologia, Inovações e...",0,https://www.gov.br/pt-br/noticias/educacao-e-p...,NaN
1,26,Pesquisa com mais de 6.000 médicos em 30 paíse...,1,https://www.aosfatos.org/noticias/e-falso-que-...,NaN
2,25,É com muita alegria que comunico que mais um p...,0,http://portal.mec.gov.br/component/content/art...,NaN
3,25,Renda Brasil unificará vários programas sociai...,0,https://agenciabrasil.ebc.com.br/politica/noti...,NaN
4,24,O Secretário-Geral da OTAN Jens Stoltenberg ta...,0,NaN,1.0
...,...,...,...,...,...
2894,1,A torcida do corona deve estar arrancando os c...,0,NaN,NaN
2895,1,“OS EUA E O CORONAVÍRUS :\n\nAcabei de assisti...,0,https://www.reuters.com/article/us-health-coro...,1.0
2896,1,Estatísticas falsas conforme depoimentos colhi...,1,NaN,1.0
2897,1,"Atenção => 🇧🇷💓💓💓 *MUITO IMPORTANTE! ""Como é qu...",0,NaN,NaN


In [ ]:
dataset_df = original_dataset_df[["text", "misinformation"]]
dataset_df

,text,misinformation
0,"O ministro da Ciência, Tecnologia, Inovações e...",0
1,Pesquisa com mais de 6.000 médicos em 30 paíse...,1
2,É com muita alegria que comunico que mais um p...,0
3,Renda Brasil unificará vários programas sociai...,0
4,O Secretário-Geral da OTAN Jens Stoltenberg ta...,0
...,...,...
2894,A torcida do corona deve estar arrancando os c...,0
2895,“OS EUA E O CORONAVÍRUS :\n\nAcabei de assisti...,0
2896,Estatísticas falsas conforme depoimentos colhi...,1
2897,"Atenção => 🇧🇷💓💓💓 *MUITO IMPORTANTE! ""Como é qu...",0


# Análise exploratória de dados

O objetivo é entender melhor e sumarizar as características dos dados, analisando quantidade e tipos de atributos, verificando distribuição do atributo alvo, identificando padrões e anomalias, removendo atributos que pareçam irrelevantes ou problemáticos, etc. Utilize gráficos e sumarizações estatísticas para a EDA. Verifique potenciais problemas nos dados, como por exemplo, a necessidade de normalizar os atributos, balancear classes, ou remover instâncias ou atributos por inconsistências nos dados.

- P1. Qual a quantidade e tipos de atributos? Existem inconsistências?
  - Quais são os atributos disponíveis?
  - Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)
  - Existem atributos que necessitam ser removidos ou transformados?
- P2. Qual a distribuição do atributo alvo?
  - Quais são as classes alvo? Qual a distribuição entre as classes? Está balanceada ou desbalanceada?


## P1. Qual a quantidade e tipos de atributos? Existem inconsistências?

### 1.1 Quais são os atributos disponíveis?

In [ ]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2899 entries, 0 to 2898
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            2898 non-null   object
 1   misinformation  2899 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.4+ KB


### 1.2 Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)

#### 1.2.1 Existem valores nulos?

In [ ]:
dataset_df[dataset_df.isnull().any(axis=1)]

,text,misinformation
847,NaN,0


Remove instância com texto nulo, pois é irrelevante para o treinamento

In [ ]:
dataset_df = dataset_df.dropna()
dataset_df.isnull().any()

text              False
misinformation    False
dtype: bool

#### 1.2.2 Existem textos que começam com URLs e podem ser removidos?

##### Busca por textos que começam com URLs

Busca instâncias de text onde começa com uma URL. Conforme Martins et al. 2021, estas instâncias podem dificultar a classificação, resultando em um ganho de aprox. 10% em F1-score ao remover estas instâncias.

In [ ]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]

/tmp/ipykernel_817797/915435192.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]


,text,misinformation
65,https://www.dfmobilidade.com.br/2020/06/covida...,0
105,https://gazetabrasil.com.br/destaques/ultimas-...,0
108,http://www.pf.gov.br/imprensa/noticias/2020/06...,0
153,https://gazetabrasil.com.br/destaques/ultimas-...,0
160,https://gazetabrasil.com.br/destaques/ultimas-...,0
...,...,...
2868,https://www.frontliner.com.br/a-maioria-das-pe...,1
2879,https://portal.fiocruz.br/coronavirus-2019-nco...,0
2881,https://economia.uol.com.br/colunas/carla-arau...,0
2886,https://revistaforum.com.br/noticias/china-pro...,0


In [ ]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]

/tmp/ipykernel_817797/915435192.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)]


,text,misinformation
65,https://www.dfmobilidade.com.br/2020/06/covida...,0
105,https://gazetabrasil.com.br/destaques/ultimas-...,0
108,http://www.pf.gov.br/imprensa/noticias/2020/06...,0
153,https://gazetabrasil.com.br/destaques/ultimas-...,0
160,https://gazetabrasil.com.br/destaques/ultimas-...,0
...,...,...
2868,https://www.frontliner.com.br/a-maioria-das-pe...,1
2879,https://portal.fiocruz.br/coronavirus-2019-nco...,0
2881,https://economia.uol.com.br/colunas/carla-arau...,0
2886,https://revistaforum.com.br/noticias/china-pro...,0


In [ ]:
dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)].count()

/tmp/ipykernel_817797/1956909523.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset_df[dataset_df['text'].str.contains(r'^(http|www)', na=False)].count()


text              604
misinformation    604
dtype: int64

#### 1.2.3 Existe o mesmo dataset filtrado conforme Martins et al. (2021)?

##### Reproduz o notebook de Martins et al. (2021)

In [ ]:
import re
dataset_df['cleanLinks'] = dataset_df['text'].apply(lambda x: re.split(r'http:\/\/.*', str(x))[0])

/tmp/ipykernel_817797/3408670061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['cleanLinks'] = dataset_df['text'].apply(lambda x: re.split(r'http:\/\/.*', str(x))[0])


In [ ]:
dataset_df['cleanLinks']

0       O ministro da Ciência, Tecnologia, Inovações e...
1       Pesquisa com mais de 6.000 médicos em 30 paíse...
2       É com muita alegria que comunico que mais um p...
3       Renda Brasil unificará vários programas sociai...
4       O Secretário-Geral da OTAN Jens Stoltenberg ta...
                              ...                        
2894    A torcida do corona deve estar arrancando os c...
2895    “OS EUA E O CORONAVÍRUS :\n\nAcabei de assisti...
2896    Estatísticas falsas conforme depoimentos colhi...
2897    Atenção => 🇧🇷💓💓💓 *MUITO IMPORTANTE! "Como é qu...
2898    [2:36 PM, 11/06/2020] Wellington: ```*ALERTA A...
Name: cleanLinks, Length: 2898, dtype: object

In [ ]:
dataset_df[dataset_df['cleanLinks'] != '' ].shape

(2776, 3)

#### 1.2.4 Existem textos que contém URLs e podem ser removidos?

##### Busca por textos que contém URLs e analisa a possibilidade de removê-los

In [ ]:
url_pattern = r'\b(?:https?://|www\.)\S+\b|\b\S+\.(com|br)\b'

no_url_df = dataset_df.copy()
no_url_df = no_url_df[~no_url_df['text'].str.contains(url_pattern, regex=True, flags=re.IGNORECASE)]
no_url_df

/tmp/ipykernel_817797/2085805525.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  no_url_df = no_url_df[~no_url_df['text'].str.contains(url_pattern, regex=True, flags=re.IGNORECASE)]


,text,misinformation,cleanLinks
32,"Gente, isso é EXTREMAMENTE GRAVE!!! Mandetta a...",1,"Gente, isso é EXTREMAMENTE GRAVE!!! Mandetta a..."
50,"Prezados amigos.. vocês sabiam que, todos os p...",1,"Prezados amigos.. vocês sabiam que, todos os p..."
54,Quando o filho de Bolsonaro culpou a China por...,1,Quando o filho de Bolsonaro culpou a China por...
56,"URGENTE, SR PRESIDENTE SALVE O BRASIL E SEU PO...",1,"URGENTE, SR PRESIDENTE SALVE O BRASIL E SEU PO..."
57,*Vamos ficar em casa e resguardar a nossa saúd...,1,*Vamos ficar em casa e resguardar a nossa saúd...
...,...,...,...
2893,"Mortes acumuladas por coronavírus no Brasil, s...",0,"Mortes acumuladas por coronavírus no Brasil, s..."
2894,A torcida do corona deve estar arrancando os c...,0,A torcida do corona deve estar arrancando os c...
2895,“OS EUA E O CORONAVÍRUS :\n\nAcabei de assisti...,0,“OS EUA E O CORONAVÍRUS :\n\nAcabei de assisti...
2896,Estatísticas falsas conforme depoimentos colhi...,1,Estatísticas falsas conforme depoimentos colhi...


In [ ]:
no_url_df.shape

(1330, 3)

#### 1.2.5 Existem textos com emojis que podem ser transformados ou removidos?

##### Busca mensagens compostas por emojis
- `emoji_count`: conta a quantidade de emojis em 'text'
- `char_count`: conta a quantidade de caracteres em 'text'
- `emoji_ratio`: calcula a taxa de emojis por mensagem

In [ ]:
def count_emojis(text):
    return sum(1 for char in text if char in emoji.EMOJI_DATA)

def char_count(text):
    return len(text)

def word_count(text):
    return len(text.split())

dataset_df['emoji_count'] = dataset_df['text'].apply(count_emojis)
dataset_df['char_count'] = dataset_df['text'].apply(char_count)
dataset_df['word_count'] = dataset_df['text'].apply(word_count)

def emoji_ratio(text):
    return count_emojis(text) / char_count(text) if char_count(text) > 0 else 0

dataset_df['emoji_ratio'] = dataset_df['text'].apply(emoji_ratio)

dataset_df

ModuleNotFoundError: No module named 'emoji'

Verifica as instâncias com maior emoji_ratio

In [ ]:
display(dataset_df.sort_values(by='emoji_ratio', ascending=False).reset_index(drop=True))

Existe alguma relação entre a taxa de emojis e o atributo preditivo?

In [ ]:
dataset_df[dataset_df['misinformation'] == 0]['emoji_ratio'].describe()

In [ ]:
dataset_df[dataset_df['misinformation'] == 1]['emoji_ratio'].describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.boxplot(x='misinformation', y='emoji_ratio', data=dataset_df, palette=['#377eb8', '#e41a1c'])
plt.title('Emoji Ratio vs Misinformation')
plt.xlabel('Misinformation')
plt.ylabel('Emoji Ratio')
plt.show()

Verifica as instâncias com maior emoji_count

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_count'], edgecolor='black')
plt.title('Histogram of emoji_count')
plt.xlabel('emoji_count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(dataset_df['emoji_count'], dataset_df['char_count'], alpha=0.5)
plt.title('Scatter Plot of Emoji Count vs Character Count')
plt.xlabel('Emoji Count')
plt.ylabel('Character Count')
plt.show()

In [ ]:
display(dataset_df.sort_values(by='emoji_count', ascending=False).reset_index(drop=True))

Não existem textos com emoji ratio maior que ~0.02. Portanto, não precisam ser tratados.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['emoji_ratio'], edgecolor='black')
plt.title('Histogram of moji_ratio')
plt.xlabel('emoji_ratio')
plt.ylabel('Frequency')

plt.show()

Existem textos com poucas palavras significativas?

In [ ]:
display(dataset_df.sort_values(by='word_count', ascending=True).reset_index(drop=True))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(dataset_df['word_count'], edgecolor='black')
plt.title('Histogram of Word Count')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

In [ ]:
dataset_df.info()

## P2. Qual a distribuição do atributo alvo?

### Experimento 1

In [ ]:
import seaborn as sns

series = dataset_df['misinformation'].value_counts()

print(series)

fig = plt.figure(figsize=(5, 3))

sns.countplot(x=dataset_df['misinformation'], data = dataset_df,
              hue='misinformation', palette=['#377eb8', '#e41a1c'],
              order=dataset_df['misinformation'].value_counts().index
)

Isto indica que o dataset está desbalanceado, fator que pode enviesar o treinamento.

### Experimento 3

In [ ]:
import seaborn as sns

series = no_url_df['misinformation'].value_counts()

print(series)

fig = plt.figure(figsize=(5, 3))

sns.countplot(x=no_url_df['misinformation'], data = no_url_df,
              hue='misinformation', palette=['#377eb8', '#e41a1c'],
              order=dataset_df['misinformation'].value_counts().index
)

# Pré-processamento

In [ ]:
  # Configurações iniciais
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Usando o dispositivo: {device}")

## Limpeza

In [ ]:
dataset_df.info()

Dataset removendo instâncias onde o texto contém URLs em seu início

In [ ]:
no_start_url_df = dataset_df[~dataset_df['text'].str.contains(r'^(http|www)', na=False)].reset_index(drop=True)
no_start_url_df.info()

Dataset removendo todas as URLs do texto

In [ ]:
no_url_df.info()

## Labels
O HuggingFace Trainer utiliza o rótulo labels para identificar os rótulos no treinamento. Renomeando a coluna alvo para 'labels'

In [ ]:
dataset_df = dataset_df.rename(columns={'misinformation': 'labels'})
no_start_url_df = no_start_url_df.rename(columns={'misinformation': 'labels'})
no_url_df = no_url_df.rename(columns={'misinformation': 'labels'})

## Tokenização

Carrega o tokenizador para `bert-base-portuguese-cased` (BERTimbau)

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer

hf_model_name = 'neuralmind/bert-base-portuguese-cased'
tokenizer = AutoTokenizer.from_pretrained(hf_model_name, do_lower_case=False)

Criamos uma função de tokenização, que será utilizada para tokenizar cada valor de um Pandas DataFrame em forma de função de mapeamento.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

In [ ]:
def tokenize(df):
  dataset = Dataset.from_pandas(df)
  dataset_tk = dataset.map(tokenize_function, batched=True, remove_columns=['text']) #'__index_level_0__'
  return dataset_tk

## Balanceamento de classes

Dado que o dataset tem sua classe misinformation desbalanceada, utilizou-se o método de cálculo de class_weights, que atribui pesos na função loss do treinador para 'compensar' o desbalanceamento.

"If "balanced", class weights will be given by `n_samples / (n_classes * np.bincount(y=labels))`. If a dictionary is given, keys are classes and values are corresponding class weights. If None is given, the class weights will be uniform."

Referências:
- https://medium.com/@heyamit10/fine-tuning-bert-for-classification-a-practical-guide-b8c1c56f252c
- https://discuss.huggingface.co/t/class-weights-for-bertforsequenceclassification/1674

get_class_weights(df):
- Cria uma instância do CrossEntropyLoss com os pesos calculados
- Recria a classe WeightedTrainer para 'sobrescrever' a classe original no HuggingFace Trainer, utilizada a computação do loss ponderada configurada acima.

In [ ]:
def get_class_weights(df):
  labels = df["labels"]

  class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)

  class_weights = torch.tensor(class_weights, dtype=torch.float)

  print(class_weights)

  loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

  class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
      
  return WeightedTrainer

In [ ]:
# Teste
get_class_weights(dataset_df)